### Cross Validation with Traget encording & Adversarial varidation

### 0. preparation

In [ ]:
from sklearn.model_selection import StratifiedKFold, GroupKFold
import lightgbm as lgb
import optuna.integration.lightgbm as lgb
import numpy as np
from sklearn.model_selection import ShuffleSplit
import pandas as pd
from matplotlib import pyplot as plt
import scipy.stats as st
import scipy.stats as stats
import seaborn as sns
import numpy as np
import random

### 1. split train/validation data

In [ ]:
# copy original train/test data
tmp_train = train.copy()
tmp_test = test.copy()


N_SPLITS = 3
SEED = 408
kf = GroupKFold(n_splits=N_SPLITS)
generator = kf.split(tmp_train, tmp_train["OBJECT"],ori_train["GROUP_K_FEATURE"])

x_tr_box = []
y_tr_box = []
x_va_box = []
y_va_box = []


# まず訓練データからテストデータのtarget_encording featureを作成
for target_col in cate_col:
    if "OBJECT" != target_col:
        for stat in ["mean","std","median","min","max","var"]:
            target_mean = ori_train.groupby(target_col)["OBJECT"].agg(stat)
            ori_test["{}_target_{}".format(target_col,stat)] = ori_test[target_col].map(target_mean).astype("float")
            

# 次にtrainとvalidationデータのtarget_encording featureを作成
for fold, (idx_train, idx_valid) in enumerate(generator):
    
    # 学習用と検証用に分ける
    x_train = ori_train.iloc[idx_train].drop("OBJECT", axis=1)
    y_train = ori_train.iloc[idx_train]["OBJECT"]
    x_valid = ori_train.iloc[idx_valid].drop("OBJECT", axis=1)
    y_valid = ori_train.iloc[idx_valid]["OBJECT"]
    
    
    # target_encording
    for target_col in cate_col:
        # 学習データを全て使って検証用のtarget_encording columnを作成
        if "OBJECT" != target_col:
            for stat in ["mean","std","median","min","max","var"]:

                data_tmp = pd.DataFrame({"tmp_col" : x_train[target_col],"OBJECT":y_train})
                target_mean = data_tmp.groupby("tmp_col")["OBJECT"].agg(stat)
                x_valid["{}_target_{}".format(target_col,stat)] = x_valid[target_col].map(target_mean).astype("float")
            
                #学習データを分割し、out-of-foldでtarget_encording columnを作成
                kf_encording = StratifiedKFold(n_splits=5)
                generator = kf_encording.split(x_train, x_train["VAIDATION_GROUP_K_FEATURE"])
                tmp = np.repeat(np.nan, x_train.shape[0])
                for fold_2, (idx_1, idx_2) in enumerate(generator):
                    
                    target_mean = data_tmp.iloc[idx_1].groupby("tmp_col")["OBJECT"].agg(stat)
                    tmp[idx_2] = x_train[target_col].iloc[idx_2].map(target_mean)

                x_train.loc[:, "{}_target_{}".format(target_col,stat)] = tmp
                
            
    x_tr_box.append(x_train)
    y_tr_box.append(y_train)
    x_va_box.append(x_valid)
    y_va_box.append(y_valid)

### 2. split train/validation data

In [ ]:
feature_ar = []

# 各validation_setでadversarial_varidationを行う
for fold in range(len(x_va_box)):

    # train : 0、test : 1
    ad_data_tr = pd.concat([x_tr_box[fold],x_va_box[fold]],axis=0).drop("OBJECT",axis=1)
    ad_data_tr["adversatial_feature"] = 0
    ad_data_te = ori_test.copy()
    ad_data_te["adversatial_feature"] = 1
    ad_data = pd.concat([ad_data_tr,ad_data_te],axis=0).sample(frac=1, random_state=0).reset_index(drop=True)
    
    
    kf = GroupKFold(n_splits=N_SPLITS)
    generator = kf.split(ad_data, ad_data["adversatial_feature"],ad_data["GROUP_K_FEATURE"])
    
    ad_data = ad_data.drop(com, axis=1)
    
    
    for fold, (idx_train, idx_valid) in enumerate(generator):
        # 学習用と検証用に分ける
        x_train = ad_data.iloc[idx_train].drop(["adversatial_feature"], axis=1)
        y_train = ad_data.iloc[idx_train]["adversatial_feature"]
        x_valid = ad_data.iloc[idx_valid].drop(["adversatial_feature"], axis=1)
        y_valid = ad_data.iloc[idx_valid]["adversatial_feature"]
        
        lgb_train = lgb.Dataset(x_train, y_train)
        lgb_valid = lgb.Dataset(x_valid, y_valid)
        model = lgb.train(
          params = {
              'objective': 'regression',
              'metric': 'rmse',
              'learning_rate': 0.01,
              'boosting': 'gbdt',
              'seed': SEED,
          },
          train_set = lgb_train,
          num_boost_round = 500,
          valid_sets = [lgb_train, lgb_valid],
          early_stopping_rounds = 10,
          verbose_eval = 50,
        )
        
        feature_ar.append(model.feature_importance())

### 3. 特徴量の重要度をランキング化

In [ ]:
# ランキングが高いほど不要な変数
fea = pd.DataFrame()
fea["col"] = x_valid.columns
fea["importance"] = np.mean(feature_ar,axis=0)
fea.sort_values(by="importance", ascending=False).head(50)